# Advent of Code 2024 Day 15 

### Setup

In [83]:
from aocd import get_data, submit

day = 15
year = 2024


In [ ]:
with open('example.txt', 'r') as file:
    raw_sample_data = "".join(file.readlines())

raw_sample_data[:100]

In [ ]:
raw_test_data = get_data(day=day, year=year)

raw_test_data[:]

##### Data Parsing

In [ ]:
import numpy as np

def parse_data(raw_data:str):
    lines = raw_data.splitlines()
    map_data = [ line for line in lines if '#' in line]
    instruction_data = [ line for line in lines if '#' not in line]

    map_list = []
    for line in map_data:
        map_list.append(list(line))
    
    map_array = np.array(map_list)
    start_pos = np.argwhere(map_array == '@')[0]

    instructions = []
    for line in instruction_data:
        for c in line:
            instructions.append(c)

    return {
        'array': map_array,
        'instructions': np.array(instructions),
        'start_pos': start_pos
    }


sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

### Part One!

In [87]:
use_sample_data = False
part = 'a'

In [ ]:
data = sample_data if use_sample_data else test_data

data

In [89]:
from typing import Literal, Union, List

Instruction = Literal['<', '>', '^', 'v']
Wall = Literal['#']
Empty = Literal['.']
Tile = Union[Wall, Empty]
Box = Literal['O']
Robot = Literal['@']
BoxHalf = Literal['[', ']']

Marker = Union[Tile, Box, Robot]

In [90]:
def get_instruction_coords(instruction: Instruction, pos: np.ndarray):
    if instruction == '^':
        return pos + np.array([-1, 0])
    if instruction == 'v':
        return pos + np.array([1, 0])
    if instruction == '<':
        return pos + np.array([0, -1])
    if instruction == '>':
        return pos + np.array([0, 1])

In [91]:
def is_box_half(marker: Marker):
    return marker in ['[', ']']

In [92]:
def get_other_box_half_marker(marker: Marker):
    if marker == '[':
        return ']'
    if marker == ']':
        return '['

In [93]:
def get_box_half_coords(pos: np.ndarray, marker: BoxHalf):
    vector = np.array([0, 1] if marker == '[' else [0, -1])
    return pos + vector

In [94]:
def perform_instruction(array: np.ndarray, pos: np.ndarray, instruction: Instruction, ignore_half_box: bool = False):
    arr = array.copy()
    marker = arr[pos[0], pos[1]]
    next_pos = get_instruction_coords(instruction, pos)
    
    if arr[next_pos[0], next_pos[1]] == '#':
        return array, pos
    
    elif arr[next_pos[0], next_pos[1]] == '.':
        arr[pos[0], pos[1]] = '.'
        arr[next_pos[0], next_pos[1]] = marker
    
    elif arr[next_pos[0], next_pos[1]] == 'O' or is_box_half(arr[next_pos[0], next_pos[1]]):
        arr, res_pos = perform_instruction(arr, next_pos, instruction)

        # no go 
        if next_pos[0] == res_pos[0] and next_pos[1] == res_pos[1]:
            return array, pos

        arr[pos[0], pos[1]] = '.'
        arr[next_pos[0], next_pos[1]] = marker
    
    if is_box_half(marker) and instruction in ['v', '^'] and not ignore_half_box:
        other_half_pos = get_box_half_coords(pos, marker)
        arr, res_pos = perform_instruction(arr, other_half_pos, instruction, ignore_half_box=True)

        # no go
        if other_half_pos[0] == res_pos[0] and other_half_pos[1] == res_pos[1]:
            return array, pos
        
        arr[other_half_pos[0], other_half_pos[1]] = '.'
        arr[res_pos[0], res_pos[1]] = get_other_box_half_marker(marker)

    return arr, next_pos
    

In [95]:
def perform_instructions(array: np.ndarray, start_pos: tuple[int, int], instructions: List[Instruction]):
    arr = array.copy()
    pos = start_pos
    for instruction in instructions:
        arr, pos = perform_instruction(arr, pos, instruction)
        
    return arr, pos

In [96]:
def calculate_gps_coords(array: np.ndarray, marker: Marker):
    indices = np.argwhere(array == marker)

    return np.sum(indices * np.array([100, 1]), axis=1)

In [ ]:
data = {k: v.copy() for k, v in data.items()}

arr, _ = perform_instructions(**data)

gps_coords = calculate_gps_coords(arr, 'O')

part_a_answer = np.sum(gps_coords)
part_a_answer

In [ ]:
if not use_sample_data and part == 'a':
    submit(answer=part_a_answer, part='a', day=day, year=year, reopen=True)

### Part Two!

In [104]:
use_sample_data = False
part='b'

In [ ]:
import numpy as np

def parse_data(raw_data:str):
    lines = raw_data.splitlines()
    map_data = [ line for line in lines if '#' in line]
    instruction_data = [ line for line in lines if '#' not in line]

    map_list = []
    for line in map_data:
        scaled_line = []
        for c in line:
            if c == '.':
                scaled_line.append('.')
                scaled_line.append('.')
            elif c == '#':
                scaled_line.append('#')
                scaled_line.append('#')
            elif c == '@':
                scaled_line.append('@')
                scaled_line.append('.')
            elif c == 'O':
                scaled_line.append('[')
                scaled_line.append(']')
                
        map_list.append(scaled_line)
    
    map_array = np.array(map_list)
    start_pos = np.argwhere(map_array == '@')[0]

    instructions = []
    for line in instruction_data:
        for c in line:
            instructions.append(c)

    return {
        'array': map_array,
        'instructions': np.array(instructions),
        'start_pos': start_pos
    }


sample_data = parse_data(raw_sample_data)
test_data = parse_data(raw_test_data)

sample_data

In [106]:
data = sample_data if use_sample_data else test_data

In [ ]:
data = {k: v.copy() for k, v in data.items()}

arr, _ = perform_instructions(**data)

gps_coords = calculate_gps_coords(arr, '[')

part_b_answer = np.sum(gps_coords)
part_b_answer

In [ ]:
if not use_sample_data and part == 'b':
    submit(answer=part_b_answer, part='b', day=day, year=year, reopen=True)